# Iterators, generator expressions, and generators

## Iterators

In [1]:
nums = [1,2,3]
iter(nums)

In [2]:
nums.__iter__()

In [3]:
nums.__reversed__()

In [4]:
it = iter(nums)
next(it)

1

In [5]:
next(it)

2

In [6]:
next(it)

3

In [7]:
# When used in a loop, StopIteration is swallowed and 
# causes the loop to finish. But with explicit invocation, 
# we can see that once the iterator is exhausted, 
# accessing it raises an exception
next(it)

StopIteration: 